In [47]:
import yfinance as yf
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
from scipy.stats import norm, t, chi2, jarque_bera
from arch import arch_model
from statsmodels.stats.diagnostic import acorr_ljungbox

In [48]:
def get_data(ticker, start, end):
    df = yf.download(ticker, start = start, end = end, interval = '1d', auto_adjust=True)
    df.index = pd.to_datetime(df.index).date
    return df['Close'], np.log(df["Close"] / df["Close"].shift(1))[1:]

In [49]:
# Normality/t check

def distribution_choice(returns, show_plots = False):
    
    if show_plots:
        plt.figure(figsize=(6,4))
        plt.hist(returns, bins=50, density=True)
        plt.title("Return Histogram")
        plt.xlabel("Return")
        plt.ylabel("Density")
        plt.show()
        
        plt.figure(figsize=(5,5))
        stats.probplot(returns.to_numpy().ravel(), dist="norm", plot=plt)
        plt.title("QQ Plot vs Normal")
        plt.show()
        jb_stat, jb_p = jarque_bera(returns)
        print(f"JB statistic: {jb_stat:.2f}")
        print(f"JB p-value: {jb_p:.4f}")
        
        df_t, loc, scale = t.fit(returns)
        plt.figure(figsize=(5,5))
        stats.probplot(returns.to_numpy().ravel(), dist=t, sparams=(df_t,), plot=plt)
        plt.title("QQ Plot vs Student-t")
        plt.show()
        print(f"t df (nu): {df_t:.2f}")
    
    mu, sigma = norm.fit(returns)
    ll_norm = np.sum(norm.logpdf(returns, mu, sigma))

    nu, loc, scale = t.fit(returns)
    ll_t = np.sum(t.logpdf(returns, nu, loc, scale))

    if ll_norm <= ll_t:
        return 't'
    else:
        return 'norm'

In [50]:
# GARCH, EWMA, Historical, Filtered Historical VaR/ES
def fit_garch(returns, distribution, show_test = False, show_results = False):
    
    returns = returns * 100
    if distribution == 't':
        garch = arch_model(
            returns,
            mean="Constant",
            vol="GARCH",
            p=1, q=1,
            dist="t"
        )

        residual = garch.fit(disp="off")
    
    else:
        garch = arch_model(
            returns,
            mean="Constant",
            vol="GARCH",
            p=1, q=1,
            dist="normal"
        )   

        residual = garch.fit(disp="off")
    
    if show_test:
        z = residual.std_resid.dropna()
        plt.figure(figsize=(12, 4))
        plt.plot(z, alpha=0.5)
        plt.axhline(0, color="black", linewidth=1)
        plt.title("Standardized Returns (GARCH)")
        plt.show() 
    
        lb_test = acorr_ljungbox(
            z**2,
            lags=(5,10,20),
            return_df=True
        )   
        print(lb_test)
    
    if show_results:
        print(residual.summary()) 

    return residual

def ewma(returns, lambd = 0.90, window = 252, show_test = False, show_plots = False):
    n = len(returns)
    sigmasq = returns.iloc[:window].var()
    sigma = np.full(n,np.nan)
    
    for t in range(window, n):
        sigmasq = lambd * sigmasq + (1 - lambd) * returns.iloc[t-1] ** 2
        sigma[t] = np.sqrt(sigmasq)
    
    sigma_series = pd.Series(sigma, index=returns.index)
    z = returns.squeeze() / sigma_series
    
    if show_test:
        plt.figure(figsize=(12, 4))
        plt.plot(z, alpha=0.5)
        plt.axhline(0, color="black", linewidth=1)
        plt.title("Standardized Returns (EWMA)")
        plt.show()  
        
        lb = acorr_ljungbox(z.dropna()**2, lags=[5, 10, 20], return_df=True)
        print(lb)
    
    if show_plots:
        fig, ax1 = plt.subplots(figsize=(12, 5))

        ax1.plot(returns, 
                color="gray", 
                alpha=0.4, 
                label="Returns")
        ax1.set_ylabel("Returns")

        ax2 = ax1.twinx()
        ax2.plot(sigma_series, 
                color="blue", 
                linewidth=2, 
                label="EWMA Vol Forecast")
        ax2.set_ylabel("Volatility")

        ax1.set_title("Returns and EWMA Volatility Forecast")

        ax1.legend(loc="upper left")
        ax2.legend(loc="upper right")

        plt.tight_layout()
        plt.show()
        
        realized_vol = returns.rolling(5).std()

        plt.figure(figsize=(12, 5))
        plt.plot(realized_vol, label="5-day Realized Vol", alpha=0.7)
        plt.plot(sigma_series, label="EWMA Vol Forecast", linewidth=2)
        plt.legend()
        plt.title("EWMA vs Realized Volatility")
        plt.show()
    
    return sigma_series, z
    
def garch_VaR(returns, alpha = 0.01):
    dist = distribution_choice(returns)
    garch = fit_garch(returns, distribution = dist)
    forecast = garch.forecast(horizon = 1)
    sigma = np.sqrt(forecast.variance.values[-1,0])
    
    if dist == 't':
        q = t.ppf(alpha, df = garch.params['nu'])
    else:
        q = norm.ppf(alpha)
    
    VaR = (garch.params['mu'] + q * sigma) / 100
    
    z = garch.std_resid.dropna().values
    qz = np.quantile(z, alpha)
    
    fhs_VaR = (garch.params['mu'] + qz * sigma) / 100
    
    return VaR, fhs_VaR, garch.params['alpha[1]'] + garch.params['beta[1]']
    
def ewma_VaR(returns, window, alpha = 0.01):
    sigma, z = ewma(returns, window = window)
    VaR = norm.ppf(alpha) * sigma
    
    fhs_VaR = pd.Series(np.nan, index=sigma.index)
    for t in range(window + 1, len(sigma)):
        z_hist = z.iloc[window:t].dropna()
        
        if len(z_hist) >= int(1/alpha):
            qz = np.quantile(z_hist, alpha)
            fhs_VaR.iloc[t] = qz * sigma.iloc[t]

    return VaR.iloc[window:], fhs_VaR.iloc[window:]
    
def historical_VaR(returns, window, alpha = 0.01):
    return returns.rolling(window).quantile(alpha)


In [51]:
# backtest
# 用不同time window来fit garch，然后rolling测试1-day VaR，对比ewma，historical，fhs。

def unconditional_independence_test(exceptions):
    exceptions = np.asarray(exceptions).astype(int)
    N00 = N01 = N10 = N11 = 0
    for t in range(1, len(exceptions)):
        if exceptions[t-1] == 0 and exceptions[t] == 0:
            N00 += 1
        elif exceptions[t-1] == 0 and exceptions[t] == 1:
            N01 += 1
        elif exceptions[t-1] == 1 and exceptions[t] == 0:
            N10 += 1
        elif exceptions[t-1] == 1 and exceptions[t] == 1:
            N11 += 1
            
    pi = (N01 + N11) / (N00 + N01 + N10 + N11)
    pi0 = N01 / (N00 + N01) if (N00 + N01) > 0 else 0
    pi1 = N11 / (N10 + N11) if (N10 + N11) > 0 else 0   
    
    def safe_log(x):
        return np.log(x) if x > 0 else 0

    logL0 = (
        (N01 + N11) * safe_log(pi)
        + (N00 + N10) * safe_log(1 - pi)
    )

    logL1 = (
        N01 * safe_log(pi0)
        + N00 * safe_log(1 - pi0)
        + N11 * safe_log(pi1)
        + N10 * safe_log(1 - pi1)
    )

    LR_ind = 2 * (logL1 - logL0)
    p_value = 1 - chi2.cdf(LR_ind, df=1)

    return {
        'pi': pi,
        "p_value": p_value
    }

def backtest(returns, window, alpha = 0.01, show_plots = True): # window: the size of data slice used for fitting
    returns = returns.squeeze()
    garch_VaRs = []
    garch_fhs = []
    alpha_beta = []
    ewma_VaRs, ewma_fhs = ewma_VaR(returns, window = window)
    historical_VaRs = historical_VaR(returns, window = window, alpha = alpha)
    
    for i in range(0, len(returns) - window ):
        rolling = returns[i: i + window]
        VaR, fhs, sum = garch_VaR(rolling, alpha = alpha)
        garch_VaRs.append(VaR)
        garch_fhs.append(fhs)
        alpha_beta.append(sum)
    
    realized = returns.iloc[window:].copy()
    df = pd.DataFrame({
        'return': realized,
        'garch_VaR': garch_VaRs,
        'garch_FHS': garch_fhs,
        'ewma_VaR': ewma_VaRs,
        'ewma_FHS': ewma_fhs,
        'historical_VaR': historical_VaRs, #.iloc[window:],
        'alpha_beta': alpha_beta
    }, index = realized.index)

    df['garch_VaR_cost'] = df['return'] - df['garch_VaR']
    df['garch_FHS_cost'] = df['return'] - df['garch_FHS']
    df['ewma_VaR_cost'] = df['return'] - df['ewma_VaR']
    df['historical_VaR_cost'] = df['return'] - df['historical_VaR']
    
    df['garch_exception_VaR'] = (df['return'] < df['garch_VaR']).astype(int)
    df['garch_exception_FHS'] = (df['return'] < df['garch_FHS']).astype(int)
    df['ewma_exception_VaR'] = (df['return'] < df['ewma_VaR']).astype(int)
    # df['ewma_exception_FHS'] = (df['return'] < df['ewma_FHS']).astype(int)
    df['historical_exception_VaR'] = (df['return'] < df['historical_VaR']).astype(int)
    
    non_nan = df[["return", "ewma_FHS"]].dropna()
    ewma_exception_FHS = (non_nan['return'] < non_nan['ewma_FHS']).astype(int)
    ewma_FHS_cost = non_nan['return'] - non_nan['ewma_FHS']
    
    '''
    plt.figure(figsize=(12,5))
    plt.plot(df.index, df["alpha_beta"], alpha=0.5, label="alpha + beta")
    plt.plot(df.index, np.log(0.5)/np.log(df["alpha_beta"]), alpha=0.5, label="half-life")
    plt.legend()
    plt.title(f"Parameter stability and sensitivity fitted on {window} days data")
    plt.ylabel("Return")
    plt.tight_layout()
    plt.show()
    '''
    
    print(f'garch VaR - average cost: ', df['garch_VaR_cost'].mean(), ', unconditional and independence test: ', unconditional_independence_test(df['garch_exception_VaR']))
    print(f'garch FHS - average cost: ', df['garch_FHS_cost'].mean(), ', unconditional and independence test: ', unconditional_independence_test(df['garch_exception_FHS']))
    print(f'ewma VaR - average cost: ', df['ewma_VaR_cost'].mean(), ', unconditional and independence test: ', unconditional_independence_test(df['ewma_exception_VaR']))
    print(f'ewma FHS - average cost: ', ewma_FHS_cost.mean(), ', unconditional and independence test: ', unconditional_independence_test(ewma_exception_FHS))
    # print(f'ewma FHS:', unconditional_independence_test(df['ewma_exception_FHS']))
    print(f'historical VaR - average cost: ', df['historical_VaR_cost'].mean(), ', unconditional and independence test: ', unconditional_independence_test(df['historical_exception_VaR']))
    
    if show_plots:
        plt.figure(figsize=(12,5))

        plt.plot(df.index, df["return"],
                color="gray", alpha=0.4, label="Returns")

        plt.plot(df.index, df["garch_VaR"],
                linewidth=1.8, label=f"GARCH VaR ({100*(1-alpha)}%)")

        plt.plot(df.index, df["garch_FHS"],
                linewidth=1.8, label=f"GARCH FHS VaR ({100*(1-alpha)}%)")
        
        plt.plot(df.index, df["ewma_VaR"],
                linewidth=1.8, label=f"EWMA VaR ({100*(1-alpha)}%)")

        plt.plot(df.index, df["ewma_FHS"],
                linewidth=1.8, label=f"EWMA FHS VaR ({100*(1-alpha)}%)")
        
        plt.plot(df.index, df["historical_VaR"],
                linewidth=1.8, label=f"Historical VaR ({100*(1-alpha)}%)")

        plt.legend()
        plt.title(f"1-day VaR Backtest: GARCH vs EWMA vs Historical vs FHS fitted on {window} days data")
        plt.ylabel("Return")
        plt.tight_layout()
        plt.show()


In [52]:
# Test on different time window
# Compare results
def comparison(ticker, start_date, end_date, window_list, test, show_plots):
    data = get_data(ticker, start_date, end_date)
    test_mean = data[1].mean().values[0]
    test_var = data[1].var().values[0]
    print('Test period average return:', test_mean, ', Test period return variance:', test_var)
    for window in window_list:
        print('----------------')
        print(f'Data window is {window}.')
        start = max(0, len(data[1])-(test + window))
        backtest(data[1][start:], window = window, show_plots = show_plots)
    
start = '2001-01-01'
end = '2009-01-01'
window_list = [500, 1000, 1500]
test = 500
show_plots = False

In [53]:
comparison('JPM', start, end, window_list, test, show_plots)

[*********************100%***********************]  1 of 1 completed

Test period average return: -1.7166248332108825e-05 , Test period return variance: 0.0007220305243941821
----------------
Data window is 500.
garch VaR - average cost:  0.09904475412530434 , unconditional and independence test:  {'pi': 0.008016032064128256, 'p_value': 0.7992961876829392}
garch FHS - average cost:  0.07736389776673781 , unconditional and independence test:  {'pi': 0.028056112224448898, 'p_value': 0.36860564925471007}
ewma VaR - average cost:  0.0714292076094289 , unconditional and independence test:  {'pi': 0.03406813627254509, 'p_value': 0.6014506645592961}
ewma FHS - average cost:  0.1055575493570588 , unconditional and independence test:  {'pi': 0.010025062656641603, 'p_value': 0.7759278859666507}
historical VaR - average cost:  0.04402897603272437 , unconditional and independence test:  {'pi': 0.052104208416833664, 'p_value': 0.7371051714513819}
----------------
Data window is 1000.
garch VaR - average cost:  0.09173269775273729 , unconditional and independence test

In [54]:
comparison('SPY', start, end, window_list, test, show_plots)

[*********************100%***********************]  1 of 1 completed

Test period average return: -0.00010668044150225898 , Test period return variance: 0.00018572976077859638
----------------
Data window is 500.


garch VaR - average cost:  0.04905234183448793 , unconditional and independence test:  {'pi': 0.008016032064128256, 'p_value': 0.7992961876829392}
garch FHS - average cost:  0.04014839297116686 , unconditional and independence test:  {'pi': 0.01603206412825651, 'p_value': 0.609636909492975}
ewma VaR - average cost:  0.034694851289731515 , unconditional and independence test:  {'pi': 0.03006012024048096, 'p_value': 0.33488866208341217}
ewma FHS - average cost:  0.05189374421329882 , unconditional and independence test:  {'pi': 0.012531328320802004, 'p_value': 0.7216608153278113}
historical VaR - average cost:  0.025901738898889595 , unconditional and independence test:  {'pi': 0.056112224448897796, 'p_value': 0.27632205878066574}
----------------
Data window is 1000.
garch VaR - average cost:  0.04325632382842051 , unconditional and independence test:  {'pi': 0.01603206412825651, 'p_value': 0.609636909492975}
garch FHS - average cost:  0.038463180265941116 , unconditional and independen

In [55]:
comparison('QQQ', start, end, window_list, test, show_plots)

[*********************100%***********************]  1 of 1 completed

Test period average return: -0.00028010959970364195 , Test period return variance: 0.00038952778958014034
----------------
Data window is 500.


garch VaR - average cost:  0.0440806348812362 , unconditional and independence test:  {'pi': 0.01002004008016032, 'p_value': 0.7503748764371779}
garch FHS - average cost:  0.038708094729513934 , unconditional and independence test:  {'pi': 0.02404809619238477, 'p_value': 0.2831221292898488}
ewma VaR - average cost:  0.03763822305637094 , unconditional and independence test:  {'pi': 0.02004008016032064, 'p_value': 0.5224640463355669}
ewma FHS - average cost:  0.046682750806992315 , unconditional and independence test:  {'pi': 0.017543859649122806, 'p_value': 0.6170657209364545}
historical VaR - average cost:  0.027835178071638707 , unconditional and independence test:  {'pi': 0.04609218436873747, 'p_value': 0.0988797935861856}
----------------
Data window is 1000.
garch VaR - average cost:  0.04074463791014606 , unconditional and independence test:  {'pi': 0.014028056112224449, 'p_value': 0.07895439065681042}
garch FHS - average cost:  0.03696926746490341 , unconditional and independenc

In [56]:
comparison('XLF', start, end, window_list, test, show_plots)

[*********************100%***********************]  1 of 1 completed

Test period average return: -0.000316805575239553 , Test period return variance: 0.00040425175382872384
----------------
Data window is 500.


garch VaR - average cost:  0.07975607069091663 , unconditional and independence test:  {'pi': 0.01002004008016032, 'p_value': 0.7503748764371779}
garch FHS - average cost:  0.06482865293711912 , unconditional and independence test:  {'pi': 0.026052104208416832, 'p_value': 0.40428091262300336}
ewma VaR - average cost:  0.059801551211668406 , unconditional and independence test:  {'pi': 0.028056112224448898, 'p_value': 0.36860564925471007}
ewma FHS - average cost:  0.08536715889068258 , unconditional and independence test:  {'pi': 0.012531328320802004, 'p_value': 0.7216608153278113}
historical VaR - average cost:  0.040114871791929636 , unconditional and independence test:  {'pi': 0.05811623246492986, 'p_value': 0.5474295974283663}
----------------
Data window is 1000.
garch VaR - average cost:  0.07401398793250422 , unconditional and independence test:  {'pi': 0.012024048096192385, 'p_value': 0.7023407829742014}
garch FHS - average cost:  0.062100698589005715 , unconditional and indepen

In [57]:
comparison('AIG', start, end, window_list, test, show_plots)

[*********************100%***********************]  1 of 1 completed

Test period average return: -0.0020196503895425484 , Test period return variance: 0.0017872275007434996
----------------
Data window is 500.


garch VaR - average cost:  0.15071182982514902 , unconditional and independence test:  {'pi': 0.014028056112224449, 'p_value': 0.07895439065681042}
garch FHS - average cost:  0.1618345176529814 , unconditional and independence test:  {'pi': 0.028056112224448898, 'p_value': 0.399326429151992}
ewma VaR - average cost:  0.1002700463420746 , unconditional and independence test:  {'pi': 0.04008016032064128, 'p_value': 0.8236039656746235}
ewma FHS - average cost:  0.22172551935520438 , unconditional and independence test:  {'pi': 0.017543859649122806, 'p_value': 0.10188167208029919}
historical VaR - average cost:  0.06312690761403325 , unconditional and independence test:  {'pi': 0.056112224448897796, 'p_value': 0.07866666586609927}
----------------
Data window is 1000.
garch VaR - average cost:  0.15721334224508132 , unconditional and independence test:  {'pi': 0.012024048096192385, 'p_value': 0.054049833370644174}
garch FHS - average cost:  0.12791741514319035 , unconditional and independe

In [58]:
comparison('GS', start, end, window_list, test, show_plots)

[*********************100%***********************]  1 of 1 completed

Test period average return: -5.3113428262713505e-05 , Test period return variance: 0.0006264776392943332
----------------
Data window is 500.


garch VaR - average cost:  0.09370641931436532 , unconditional and independence test:  {'pi': 0.006012024048096192, 'p_value': 0.8489167388721746}
garch FHS - average cost:  0.06560825293093424 , unconditional and independence test:  {'pi': 0.028056112224448898, 'p_value': 0.36860564925471007}
ewma VaR - average cost:  0.07123067357823329 , unconditional and independence test:  {'pi': 0.014028056112224449, 'p_value': 0.6553724466331007}
ewma FHS - average cost:  0.08352809370540347 , unconditional and independence test:  {'pi': 0.012531328320802004, 'p_value': 0.7216608153278113}
historical VaR - average cost:  0.04879546363796776 , unconditional and independence test:  {'pi': 0.05410821643286573, 'p_value': 0.6533264109243839}
----------------
Data window is 1000.
garch VaR - average cost:  0.0854537850546719 , unconditional and independence test:  {'pi': 0.008016032064128256, 'p_value': 0.7992961876829392}
garch FHS - average cost:  0.0651103829408481 , unconditional and independence

In [59]:
comparison('MS', start, end, window_list, test, show_plots)

[*********************100%***********************]  1 of 1 completed

Test period average return: -0.0005771403944806545 , Test period return variance: 0.0012741988541369153
----------------
Data window is 500.


garch VaR - average cost:  0.13141672378299593 , unconditional and independence test:  {'pi': 0.014028056112224449, 'p_value': 0.6553724466331007}
garch FHS - average cost:  0.11939868917606986 , unconditional and independence test:  {'pi': 0.022044088176352707, 'p_value': 0.2319141005208899}
ewma VaR - average cost:  0.09521924908620401 , unconditional and independence test:  {'pi': 0.026052104208416832, 'p_value': 0.3390520829437661}
ewma FHS - average cost:  0.1500474985101461 , unconditional and independence test:  {'pi': 0.015037593984962405, 'p_value': 0.6686262651624997}
historical VaR - average cost:  0.06193793796610045 , unconditional and independence test:  {'pi': 0.05410821643286573, 'p_value': 0.0019051838388645104}
----------------
Data window is 1000.
garch VaR - average cost:  0.1222010003070773 , unconditional and independence test:  {'pi': 0.018036072144288578, 'p_value': 0.5652878645357801}
garch FHS - average cost:  0.11237633228896278 , unconditional and independen

In [60]:
comparison('BAC', start, end, window_list, test, show_plots)

[*********************100%***********************]  1 of 1 completed


Test period average return: -7.086552864749135e-05 , Test period return variance: 0.0006778083754004277
----------------
Data window is 500.
garch VaR - average cost:  0.10403158013923723 , unconditional and independence test:  {'pi': 0.008016032064128256, 'p_value': 0.7992961876829392}
garch FHS - average cost:  0.08170052749555301 , unconditional and independence test:  {'pi': 0.026052104208416832, 'p_value': 0.40428091262300336}
ewma VaR - average cost:  0.07376744012237925 , unconditional and independence test:  {'pi': 0.03206412825651302, 'p_value': 0.3031615095746394}
ewma FHS - average cost:  0.10722499152817751 , unconditional and independence test:  {'pi': 0.010025062656641603, 'p_value': 0.7759278859666507}
historical VaR - average cost:  0.04166763064215672 , unconditional and independence test:  {'pi': 0.07214428857715431, 'p_value': 0.7919148559143389}
----------------
Data window is 1000.
garch VaR - average cost:  0.09757012280014334 , unconditional and independence test

In [61]:
comparison('C', start, end, window_list, test, show_plots)

[*********************100%***********************]  1 of 1 completed

Test period average return:

 -0.0008363464525531004 , Test period return variance: 0.0009255531787661465
----------------
Data window is 500.
garch VaR - average cost:  0.12004735133785621 , unconditional and independence test:  {'pi': 0.01002004008016032, 'p_value': 0.7503748764371779}
garch FHS - average cost:  0.08688477091066071 , unconditional and independence test:  {'pi': 0.02404809619238477, 'p_value': 0.027588856777898907}
ewma VaR - average cost:  0.08261302956795792 , unconditional and independence test:  {'pi': 0.03006012024048096, 'p_value': 0.07328921995108151}
ewma FHS - average cost:  0.12529330567748492 , unconditional and independence test:  {'pi': 0.012531328320802004, 'p_value': 0.7216608153278113}
historical VaR - average cost:  0.05046422580875633 , unconditional and independence test:  {'pi': 0.06412825651302605, 'p_value': 0.968883117937453}
----------------
Data window is 1000.
garch VaR - average cost:  0.11596887904350663 , unconditional and independence test:  {'pi': 0.0100200400801603